In [1]:
import numpy as np
import pandas as pd

In [97]:
df= pd.read_csv('new.csv')

In [98]:
df.corr()

,No. of FTE Request Raised
No. of FTE Request Raised,1.0


In [99]:
df.head()

,Month DD Raised,No. of FTE Request Raised,SkillList,Location,Experience Grade,Practice,Skill Group,Demand Source
0,May,4,Salesforce (Functional),Bangalore,A4,BANK,(F) IT Operations,Account
1,October,2,"Amazon Web services - Build , Administration a...",Bhubaneswar,B2,M&FT,(F) IT Operations,Account
2,March,2,"Amazon Web services - Build , Administration a...",Pune,A3,TEST,NaN,Support
3,October,1,Test Automation,Bhubaneswar,B2,M&FT,(F) IT Operations,Account
4,June,2,PMO (Account),Noida,A5,I&D,(D) Banking Sector,Proactive


In [100]:
df.describe()

,No. of FTE Request Raised
count,1023.000000
mean,2.482893
std,1.118012
min,1.000000
25%,2.000000
50%,2.000000
75%,4.000000
max,4.000000


In [101]:
df.columns = ['Month', 'Requests', 'Skill', 'Location', 'Exp' , 'Practice' , 'SkillGroup', 'Demand']

In [102]:
df.head()

,Month,Requests,Skill,Location,Exp,Practice,SkillGroup,Demand
0,May,4,Salesforce (Functional),Bangalore,A4,BANK,(F) IT Operations,Account
1,October,2,"Amazon Web services - Build , Administration a...",Bhubaneswar,B2,M&FT,(F) IT Operations,Account
2,March,2,"Amazon Web services - Build , Administration a...",Pune,A3,TEST,NaN,Support
3,October,1,Test Automation,Bhubaneswar,B2,M&FT,(F) IT Operations,Account
4,June,2,PMO (Account),Noida,A5,I&D,(D) Banking Sector,Proactive


In [103]:
df.nunique()

Month           8
Requests        4
Skill         222
Location        8
Exp             8
Practice       11
SkillGroup     52
Demand          3
dtype: int64

In [104]:
df['Month'].unique()
months = {'May':5, 'October':10, 'March':3, 'June':6, 'August':8, 'February':2, 'January':1,
       'December':12}

In [105]:
df.Month = df.Month.map(months)

In [106]:
df.sort_values('Month', inplace=True)

In [107]:
df.head()

,Month,Requests,Skill,Location,Exp,Practice,SkillGroup,Demand
409,1,2,Testing Manual,Chennai,A1,DCX,(T) Microsoft Development Technologies,Proactive
356,1,4,"Selenium,Test Automation,Cucumber, Behat,Java 7",Chennai,A1,DCX,(T) Guidewire,Account
166,1,4,CRM - Salesforce.com,Chennai,A1,DCX,(T) Mainframe Development Technologies,Proactive
500,1,1,Teradata,Chennai,A1,DCX,(F) IT Consulting,Proactive
909,1,1,"Amazon Web services - Build , Administration a...",Chennai,A1,DCX,(T) Web Development Technologies,Proactive


In [108]:

df.Month = df.Month.astype(str)

In [109]:
df.isnull().values.any()

True

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1023 entries, 409 to 543
Data columns (total 8 columns):
Month         1023 non-null object
Requests      1023 non-null int64
Skill         1023 non-null object
Location      1023 non-null object
Exp           1023 non-null object
Practice      1023 non-null object
SkillGroup    981 non-null object
Demand        1023 non-null object
dtypes: int64(1), object(7)
memory usage: 71.9+ KB


In [111]:
df.loc[df['SkillGroup'].isnull() , 'SkillGroup'] = 'Undefined'

In [112]:
y = np.array(df['Requests'])
y.shape

(1023,)

In [89]:
df_X = df.drop(columns='Requests', axis=1)

In [90]:
df_new = pd.get_dummies(df_X)

In [91]:
df_new.shape

(1023, 313)

In [113]:
eda = pd.concat(df_new, df['Requests'])
eda.corr()

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [92]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(df_new), y , test_size = 0.05)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((971, 313), (52, 313), (971,), (52,))

In [93]:
import keras
from keras.layers import BatchNormalization
from keras.layers.core import Flatten,Dense,Dropout, Activation, Lambda
from keras.models import Model, Sequential
from keras.optimizers import Adam,SGD
from keras.callbacks import TensorBoard

In [94]:
model = Sequential()
#model.add(Lambda(lambda x: x , input_shape = (24)))
#model.add(Lambda(lambda x: x+0.1 , input_shape = (64,64,3)))
#model.add(Flatten())
#model.add(BatchNormalization(input_shape=(313,)))
model.add(Dense(1000,input_shape=(313,)))
model.add(Activation('elu'))


model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(500))
model.add(Activation('elu'))
model.add(Dropout(0.2))
#model.add(BatchNormalization())
model.add(Dense(50))
model.add(Activation('elu'))

model.add(Dense(1))
model.add(Activation('linear'))
model.compile(optimizer = 'Adam' , loss = 'mse', metrics=['accuracy'])

In [95]:
model.fit(X_train, y_train, epochs=50, verbose =1, validation_data=(X_test, y_test), batch_size =64)

Train on 971 samples, validate on 52 samples
Epoch 1/50
971/971 [==============================] - 1s 1ms/step - loss: 2.1382 - acc: 0.2626 - val_loss: 1.0493 - val_acc: 0.4231
Epoch 2/50
971/971 [==============================] - 0s 175us/step - loss: 1.3254 - acc: 0.2678 - val_loss: 1.0127 - val_acc: 0.3462
Epoch 3/50
971/971 [==============================] - 0s 178us/step - loss: 1.2199 - acc: 0.2832 - val_loss: 1.1989 - val_acc: 0.3654
Epoch 4/50
971/971 [==============================] - 0s 178us/step - loss: 1.1823 - acc: 0.2842 - val_loss: 1.1394 - val_acc: 0.3077
Epoch 5/50
971/971 [==============================] - 0s 182us/step - loss: 1.1333 - acc: 0.3110 - val_loss: 1.1105 - val_acc: 0.4231
Epoch 6/50
971/971 [==============================] - 0s 180us/step - loss: 1.0016 - acc: 0.3326 - val_loss: 1.1400 - val_acc: 0.3846
Epoch 7/50
971/971 [==============================] - 0s 180us/step - loss: 1.0035 - acc: 0.3450 - val_loss: 1.1566 - val_acc: 0.3077
Epoch 8/50
971/971 